In [ ]:
# 関連ライブラリのimport
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
#fold数
folds = 5

import pickle
import os
import gc
gc.enable()
import warnings
warnings.filterwarnings("ignore")
import time

import pandas as pd

from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
import re
import matplotlib.pyplot as plt
import numpy as np

import matplotlib
import japanize_matplotlib
import seaborn as sns

import mojimoji
import itertools

In [ ]:
train = pd.read_csv("data/train_processed_add_geocoding_meshcode_crime_landprice.csv")
test = pd.read_csv("data/test_processed_add_geocoding_meshcode_crime_landprice.csv")

In [ ]:
train_and_test = pd.concat([train,test],axis=0)

In [ ]:
train_and_test["keiyaku_length"] = train_and_test["keiyaku_length"].fillna(train_and_test["keiyaku_length"].mean())
train["keiyaku_length"] = train["keiyaku_length"].fillna(train["keiyaku_length"].mean())

In [ ]:
exclusion = ["所在地", "アクセス", "番地", "tokens"] + ["id", "賃料", "lat", "lng"]

In [ ]:
one_hot_feats = [
 '24時間換気システム',
 '2面採光',
 '3面採光',
 'ウォークインクローゼット',
 'エアコン付',
 'エレベーター',
 'オール電化',
 'ガスその他',
 'ガス暖房',
 'クッションフロア',
 'シューズボックス',
 'タイル張り',
 'トランクルーム',
 'バリアフリー',
 'バルコニー',
 'フローリング',
 'プロパンガス',
 'ペアガラス',
 'ルーフバルコニー',
 'ロフト付き',
 '下水',
 '二世帯住宅',
 '二重サッシ',
 '井戸',
 '公営水道',
 '冷房',
 '出窓',
 '地下室',
 '室内洗濯機置場',
 '室外洗濯機置場',
 '専用庭',
 '床下収納',
 '床暖房',
 '排水その他',
 '敷地内ごみ置き場',
 '水道その他',
 '汲み取り',
 '洗濯機置場なし',
 '浄化槽',
 '石油暖房',
 '都市ガス',
 '防音室',
 'IHコンロ',
 'L字キッチン',
 'カウンターキッチン',
 'ガスコンロ',
 'コンロ1口',
 'コンロ2口',
 'コンロ3口',
 'コンロ4口以上',
 'コンロ設置可(コンロ1口)',
 'コンロ設置可(コンロ2口)',
 'コンロ設置可(コンロ3口)',
 'コンロ設置可(コンロ4口以上)',
 'コンロ設置可(口数不明)',
 'システムキッチン',
 '冷蔵庫あり',
 '独立キッチン',
 '給湯',
 '電気コンロ',
 'BSアンテナ',
 'CATV',
 'CSアンテナ',
 'インターネット使用料無料',
 'インターネット対応',
 '光ファイバー',
 '有線放送',
 '高速インターネット',
'シャワー',
 'トイレなし',
 'バスなし',
 'バス・トイレ別',
 '共同トイレ',
 '共同バス',
 '専用トイレ',
 '専用バス',
 '洗面台独立',
 '浴室乾燥機',
 '温水洗浄便座',
 '脱衣所',
 '追焚機能',
'K有無',
 'D有無',
 'L有無',
 'S有無',
'定期借家フラグ',
'park_bike',
 'park_motor',
 'park_car']

In [ ]:
categorical_feats_name = ['方角','建物構造','区', "railway"]

In [ ]:
columns = [n for n in list(train_and_test.columns) if n not in one_hot_feats and n not in categorical_feats_name and n not in exclusion]

In [ ]:
columns

In [ ]:
def outlier(series):
    q1 = series.quantile(.25)
    q3 = series.quantile(.75)
    iqr = q3 - q1
    iqr_rate = q3 / q1
    bias=1.5
    #外れ値の基準点
    outlier_min = q1 - (iqr) * bias
    outlier_max = q3 + (iqr) * bias
    return iqr, iqr_rate, outlier_min, outlier_max

In [ ]:
outlier(train_and_test["area"])

In [ ]:
log_feature = np.log1p(train_and_test[columns])
c = log_feature.columns
log_feature.columns = [n+"_log" for n in c]
log_feature
train_and_test = pd.concat([train_and_test, log_feature], axis=1)

In [ ]:
# c
# groupby_df = train_and_test.groupby(c).mean().reset_index()[columns.remove(c)]
# groupby_df
# pd.merge(train_and_test[c], groupby_df)

In [ ]:
train_and_test["公示価格/面積"] = train_and_test["公示価格"] / train_and_test["area"]
train_and_test["面積/部屋数"] = train_and_test["area"] / train_and_test["部屋数"]
train_and_test["高さ率"] = train_and_test["floor"] / train_and_test["all_floor"]
train_and_test["面積*高さ"] = train_and_test["area"] * train_and_test["floor"]

# train_and_test["公示価格*面積"] = train_and_test["公示価格"] * train_and_test["area"]

train_and_test["公示価格_bin"] = pd.cut(train_and_test["公示価格"], 5, labels=False)

In [ ]:
train_and_test["所在地_v2"] = train_and_test["所在地"].str.replace("丁目.*", "")+"丁目"
location = train_and_test["所在地_v2"].str.replace("東京都","").str.split("区", expand=True).rename(columns={0:"区", 1:"丁目"})
labels, uniques = pd.factorize(location['丁目'])
train_and_test['丁目'] = labels
train_and_test = train_and_test.drop("所在地_v2", axis=1)

In [ ]:
train_and_test["間取り"] = train_and_test["部屋数"]*1000 + train_and_test["L有無"]*100 + train_and_test["D有無"]*10 + train_and_test["K有無"]*1 + train_and_test["S有無"]*1

In [ ]:
train_and_test["間取り"]

In [ ]:
def make_groupby_from(df, groupby_col, agg_func, col):
    groupby_df = df.groupby(groupby_col).agg({col:agg_func}).reset_index().fillna(0)
    agg_func_column = [f"{agg}_{'&'.join(groupby_col)}" for agg in agg_func]
    groupby_df.columns = groupby_col + agg_func_column
    return groupby_df

In [ ]:
cols = [
    '建物構造',
    '区',
    '丁目',
    '間取り'
]



agg_func = ["mean", "median", "min", "max", "std", "skew"]


for c in cols:
    print(c)
    groupby_col = c
    groupby_df = train_and_test.groupby(groupby_col).agg({"age":"mean", "area":"mean", "all_floor":"mean"}).reset_index().fillna(0)
    groupby_df.columns = [groupby_col] + [f"mean_age_{c}", f"mean_area_{c}", f"mean_all_floor_{c}"]
    train_and_test = pd.merge(train_and_test, groupby_df, on=groupby_col, how="left")
    train_and_test[f"area_平均倍率_{c}"] = train_and_test["area"] / train_and_test[f"mean_area_{c}"]
    train_and_test[f"area_平均差分_{c}"] = train_and_test["area"] - train_and_test[f"mean_area_{c}"]
    train_and_test[f"age_平均倍率_{c}"] = train_and_test["age"] / train_and_test[f"mean_age_{c}"]
    train_and_test[f"age_平均差分_{c}"] = train_and_test["age"] - train_and_test[f"mean_age_{c}"]
    train_and_test[f"all_floor_平均倍率_{c}"] = train_and_test["all_floor"] / train_and_test[f"mean_all_floor_{c}"]
    train_and_test[f"all_floor_平均差分_{c}"] = train_and_test["all_floor"] - train_and_test[f"mean_all_floor_{c}"]

    groupby_df = train_and_test.groupby(groupby_col).agg({f"area_平均倍率_{c}":agg_func, f"area_平均差分_{c}":agg_func,\
                                                          f"age_平均倍率_{c}":agg_func, f"age_平均差分_{c}":agg_func,\
                                                          f"all_floor_平均倍率_{c}":agg_func, f"all_floor_平均差分_{c}":agg_func}).reset_index().fillna(0)
    agg_func_column = [f"{g}_{n}_{''.join(groupby_col)}_{agg}" for g in ["area", "age", "all_floor"] for n in ["平均倍率", "平均差分"] for agg in agg_func]
    groupby_df.columns = [groupby_col] + agg_func_column
    train_and_test = pd.merge(train_and_test, groupby_df, on=groupby_col, how="left")

# 追加

In [ ]:
drop_columns = ['mean_age_町',
 'mean_area_町',
 'mean_all_floor_町',
 'area_平均倍率_町',
 'area_平均差分_町',
 'age_平均倍率_町',
 'age_平均差分_町',
 'all_floor_平均倍率_町',
 'all_floor_平均差分_町',
 'area_平均倍率_町_mean',
 'area_平均倍率_町_median',
 'area_平均倍率_町_min',
 'area_平均倍率_町_max',
 'area_平均倍率_町_std',
 'area_平均倍率_町_skew',
 'area_平均差分_町_mean',
 'area_平均差分_町_median',
 'area_平均差分_町_min',
 'area_平均差分_町_max',
 'area_平均差分_町_std',
 'area_平均差分_町_skew',
 'age_平均倍率_町_mean',
 'age_平均倍率_町_median',
 'age_平均倍率_町_min',
 'age_平均倍率_町_max',
 'age_平均倍率_町_std',
 'age_平均倍率_町_skew',
 'age_平均差分_町_mean',
 'age_平均差分_町_median',
 'age_平均差分_町_min',
 'age_平均差分_町_max',
 'age_平均差分_町_std',
 'age_平均差分_町_skew',
 'all_floor_平均倍率_町_mean',
 'all_floor_平均倍率_町_median',
 'all_floor_平均倍率_町_min',
 'all_floor_平均倍率_町_max',
 'all_floor_平均倍率_町_std',
 'all_floor_平均倍率_町_skew',
 'all_floor_平均差分_町_mean',
 'all_floor_平均差分_町_median',
 'all_floor_平均差分_町_min',
 'all_floor_平均差分_町_max',
 'all_floor_平均差分_町_std',
 'all_floor_平均差分_町_skew',
               'meshcode_賃料mean']

In [ ]:
train = pd.read_csv("data/train_processed_add_geocoding_meshcode_crime_landprice_ladmark_log_groupby.csv")
test = pd.read_csv("data/test_processed_add_geocoding_meshcode_crime_landprice_ladmark_log_groupby.csv")
train_and_test = pd.concat([train,test],axis=0)

In [ ]:
train_and_test["賃料"] = train_and_test["賃料"].replace(1203500, 123500)

In [ ]:
labels, uniques = pd.factorize(train_and_test['町'])
train_and_test['町'] = labels

In [ ]:
train_and_test = train_and_test.drop(drop_columns, axis=1)

In [ ]:
cols = [
    "町"
]



agg_func = ["mean", "median", "min", "max", "std", "skew"]


for c in cols:
    print(c)
    groupby_col = c
    groupby_df = train_and_test.groupby(groupby_col).agg({"age":"mean", "area":"mean", "all_floor":"mean"}).reset_index().fillna(0)
    groupby_df.columns = [groupby_col] + [f"mean_age_{c}", f"mean_area_{c}", f"mean_all_floor_{c}"]
    train_and_test = pd.merge(train_and_test, groupby_df, on=groupby_col, how="left")
    train_and_test[f"area_平均倍率_{c}"] = train_and_test["area"] / train_and_test[f"mean_area_{c}"]
    train_and_test[f"area_平均差分_{c}"] = train_and_test["area"] - train_and_test[f"mean_area_{c}"]
    train_and_test[f"age_平均倍率_{c}"] = train_and_test["age"] / train_and_test[f"mean_age_{c}"]
    train_and_test[f"age_平均差分_{c}"] = train_and_test["age"] - train_and_test[f"mean_age_{c}"]
    train_and_test[f"all_floor_平均倍率_{c}"] = train_and_test["all_floor"] / train_and_test[f"mean_all_floor_{c}"]
    train_and_test[f"all_floor_平均差分_{c}"] = train_and_test["all_floor"] - train_and_test[f"mean_all_floor_{c}"]

    groupby_df = train_and_test.groupby(groupby_col).agg({f"area_平均倍率_{c}":agg_func, f"area_平均差分_{c}":agg_func,\
                                                          f"age_平均倍率_{c}":agg_func, f"age_平均差分_{c}":agg_func,\
                                                          f"all_floor_平均倍率_{c}":agg_func, f"all_floor_平均差分_{c}":agg_func}).reset_index().fillna(0)
    agg_func_column = [f"{g}_{n}_{''.join(groupby_col)}_{agg}" for g in ["area", "age", "all_floor"] for n in ["平均倍率", "平均差分"] for agg in agg_func]
    groupby_df.columns = [groupby_col] + agg_func_column
    train_and_test = pd.merge(train_and_test, groupby_df, on=groupby_col, how="left")

In [ ]:
categorical_feats_name = ['方角','建物構造','区', "railway", "駅名", "町"]
drop_columns = [c+"_賃料mean" for c in categorical_feats_name]

In [ ]:
train_and_test = train_and_test.drop(drop_columns, axis=1)

In [ ]:
from sklearn.model_selection import KFold
categorical_feats_name = ['方角','建物構造','区', "railway", "駅名", "町"]

# クロスバリデーションのfoldを定義
kf = KFold(n_splits=4, shuffle=True, random_state=71)
train_and_test[categorical_feats_name] = train_and_test[categorical_feats_name].fillna(-1)
# 変数をループしてtarget encoding
for c in categorical_feats_name:

    # targetを付加
    data_tmp = pd.DataFrame({c: train_and_test[c], 'target': train_and_test["賃料"]})
    # 変換後の値を格納する配列を準備
    tmp = np.repeat(np.nan, train_x.shape[0])

    # 学習データからバリデーションデータを分ける
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_and_test)):
        # 学習データについて、各カテゴリにおける目的変数の平均を計算
        target_mean = data_tmp.iloc[tr_idx].groupby(c)['target'].mean()
        # バリデーションデータについて、変換後の値を一時配列に格納
        tmp[va_idx] = train_and_test[c].iloc[va_idx].map(target_mean)

    # 変換後のデータで元の変数を置換
    train_and_test[c+"_賃料mean"] = tmp

In [ ]:
train = train_and_test[train_and_test['賃料'].notnull()].copy().reset_index(drop=True)
test = train_and_test[train_and_test['賃料'].isnull()].copy().reset_index(drop=True)
del train_and_test

train.to_csv("data/train_processed_add_geocoding_meshcode_crime_landprice_ladmark_log_groupby.csv", index=False)
test.to_csv("data/test_processed_add_geocoding_meshcode_crime_landprice_ladmark_log_groupby.csv", index=False)

gc.collect()

In [ ]:
print(train.shape,test.shape)